# 01 - Agentic RAG Medical Assistant

## Requirements
These are the required _Libraries_ and _Environment Variables_ for this notebook


### Libraries Required


In [ ]:
# for setting up Jupyter widgets and notebook features
%conda install conda-forge::ipywidgets --update-deps --force-reinstall
%conda install conda-forge::ipykernel --update-deps --force-reinstall


- [LangSmith](https://docs.langchain.com/langsmith/home)
- [LangChain](https://reference.langchain.com/python/langchain/langchain)
- [LangChain Core](https://reference.langchain.com/python/langchain_core)
- [LangChain Groq Model Provider](https://reference.langchain.com/python/integrations/langchain_groq)
- [Chroma Vector Database](https://docs.trychroma.com/docs/overview/introduction)
- [Sentence Transformers](https://www.sbert.net)

In [ ]:
%conda install conda-forge::langsmith
%conda install conda-forge::langchain --update-deps --force-reinstall
%conda install conda-forge::langchain-core --update-deps --force-reinstall
%conda install conda-forge::langchain-groq

%conda install conda-forge::chromadb

%conda install conda-forge::pyarrow
%conda install conda-forge::datasets
%conda install conda-forge::sentence-transformers


### Variables Required

| Token Name                     | `.env` Name          | Where to Get / Setting Value                                                      |                                                                                              Reference |
| :----------------------------- | :------------------- | :-------------------------------------------------------------------------------- | -----------------------------------------------------------------------------------------------------: |
| Groq API Key                   | `GROQ_API_KEY`       | [Groq Console](https://console.groq.com/keys)                                     |                                              [Groq API Docs](https://console.groq.com/docs/quickstart) |
| LangSmith API Key              | `LANGSMITH_API_KEY`  | [LangSmith Settings](https://smith.langchain.com/settings)                        |                                   [LangSmith API Reference](https://docs.langchain.com/langsmith/home) |
| LangSmith Tracing              | `LANGSMITH_TRACING`  | A Boolean, set the value to `true` or `false` to enable or disable logging traces | [LangSmith Observability API Reference](https://docs.langchain.com/langsmith/observability-quickstart) |
| LangSmith Endpoint             | `LANGSMITH_ENDPOINT` | The LangSmith Endpoint to log the traces (<https://api.smith.langchain.com>)      | [LangSmith Observability API Reference](https://docs.langchain.com/langsmith/observability-quickstart) |
| LangSmith Project Name         | `LANGSMITH_PROJECT`  | The name of the project to log the traces under                                   | [LangSmith Observability API Reference](https://docs.langchain.com/langsmith/observability-quickstart) |
| Hugging Face User Access Token | `HF_TOKEN`           | [Hugging Face Settings](https://huggingface.co/settings/tokens)                   |                                                       [Hugging Face Docs](https://huggingface.co/docs) |


In [ ]:
from pathlib import Path
import sys

ROOT = Path().resolve().parent.parent
sys.path.append(str(ROOT))


In [ ]:
from utils import (
    set_env_variables,
)

ENV_FILE = ROOT / ".env"

set_env_variables(env_file=ENV_FILE)


## Actual Shenanigans


In [1]:
from langchain.agents import create_agent
from langchain.tools import tool
from langchain_groq import ChatGroq
from langchain.messages import HumanMessage
from langchain_text_splitters import RecursiveCharacterTextSplitter

from sentence_transformers import SentenceTransformer

import chromadb


In [2]:
documents = [
    {
        "title": "Aspirin for Cardiovascular Disease Prevention",
        "source": "Journal of Cardiology 2024",
        "content": """
        Aspirin is a nonsteroidal anti-inflammatory drug (NSAID) commonly used
        for pain relief, fever reduction, and inflammation management. Recent
        studies show that low-dose aspirin (75-100 mg daily) may reduce
        cardiovascular events in high-risk patients.

        Efficacy: 85% reduction in secondary heart attacks when used appropriately.
        Side effects: GI upset (20%), increased bleeding risk (5%), rare severe allergic reactions.
        Contraindications: Active bleeding, severe liver disease, aspirin allergy.

        When combined with other medications, particularly other NSAIDs, bleeding
        risk increases significantly. Always consult with a healthcare provider
        before starting aspirin therapy.
        """,
    },
    {
        "title": "Ibuprofen Drug Interactions and Safety",
        "source": "FDA Safety Update 2024",
        "content": """
        Ibuprofen is an NSAID used for moderate pain and inflammation. It works
        by inhibiting prostaglandin synthesis. Common dosing is 200-400 mg
        every 4-6 hours.

        Serious interactions: NEVER combine with aspirin (risk of severe GI bleeding),
        warfarin (increased bleeding), or other NSAIDs.

        Renal concerns: Ibuprofen can impair kidney function, especially in elderly
        patients or those with existing renal disease. Baseline renal function
        testing recommended.

        The FDA has strengthened warnings about cardiovascular and gastrointestinal risks.
        Long-term use should be avoided unless under medical supervision.
        """,
    },
    {
        "title": "Pain Management Alternatives to NSAIDs",
        "source": "Pain Management Journal 2024",
        "content": """
        Given the risks associated with NSAIDs, alternative pain management
        strategies are increasingly recommended. Options include:

        1. Acetaminophen: Safer for most patients but hepatotoxic at high doses.
        2. Topical NSAIDs: Lower systemic absorption reduces side effects.
        3. Physical therapy: Effective for chronic pain without medication risks.
        4. Newer agents: COX-2 selective inhibitors have improved safety profiles.
        5. Multimodal approach: Combining multiple strategies for synergistic effects.

        Evidence suggests that multimodal pain management reduces NSAID
        dependence by 60%.
        """,
    },
    {
        "title": "Emergency Management of NSAID Overdose",
        "source": "Toxicology Review 2024",
        "content": """
        NSAID overdose or toxicity presents with gastrointestinal bleeding, renal
        dysfunction, and in severe cases, metabolic acidosis and CNS effects.

        Treatment: Supportive care, GI decontamination (within 4 hours), monitoring
        of electrolytes and renal function. Antacids or H2 blockers may reduce
        GI symptoms.

        For severe cases: Hospitalization required. Hemodialysis may be needed
        for certain NSAIDs with prolonged half-lives.

        Prognosis: Most patients recover fully with early intervention. Mortality
        is rare unless accompanied by other overdoses or severe comorbidity.
        """,
    },
]


### Initializing Chroma Collection

Initializing the

- [BAAI BGE-SMALL Embedding model](https://huggingface.co/BAAI/bge-small-en-v1.5)
- Chroma [Ephemeral Client](https://docs.trychroma.com/docs/run-chroma/ephemeral-client) and [Collection](https://docs.trychroma.com/docs/collections/manage-collections#embedding-functions) using the BGE embedding model


In [ ]:
embedding_model = SentenceTransformer("BAAI/bge-small-en-v1.5")

chroma_client = chromadb.EphemeralClient()

collection = chroma_client.get_or_create_collection(
    name="medical_documents_v1", metadata={"hnsw:space": "cosine"}
)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300, chunk_overlap=50, separators=["\n\n", ".", " "]
)
chunk_counter = 0
for doc in documents:
    chunks = text_splitter.split_text(doc["content"])
    print(f"  '{doc['title']}' → {len(chunks)} chunks")

    for chunk_idx, chunk in enumerate(chunks):
        embedding = embedding_model.encode([chunk])[0].tolist()

        collection.add(
            ids=[f"doc_{chunk_counter}"],
            documents=[chunk],
            embeddings=[embedding],
            metadata=[
                {
                    "source": doc["source"],
                    "title": doc["title"],
                    "chunk_index": chunk_idx,
                }
            ],
        )

        chunk_counter += 1
print(f"Total chunks added to ChromaDB: {chunk_counter}")


model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

### Defining Retrieval Tool

In [ ]:
@tool
def search_documents(query: str, top_k: int = 3) -> str:
    """Search the medical document database. Use query parameter for your search term and top_k for number of results (default 3)."""
    try:
        query_embedding = embedding_model.encode([query])[0].tolist()

        results = collection.query(
            query_embeddings=[query_embedding], n_results=top_k if top_k > 0 else 3
        )

        if not results["documents"] or len(results["documents"][0]) == 0:
            return "No relevant documents found. Try a different query."

        formatted_results = []
        for i, (document, metadata, distance) in enumerate(
            zip(
                results["documents"][0],
                results["metadata"][0],
                results["distances"][0],
            )
        ):
            similarity_score = 1 - distance
            formatted_results.append(
                f"\n--- Result {i + 1} (Relevance: {similarity_score:.2%}) ---\n"
                f"Source: {metadata['title']} ({metadata['source']})\n"
                f"Content: {document}\n"
            )

        return "".join(formatted_results)

    except Exception as e:
        return f"Error searching documents: {str(e)}. Please try again."


### Initializing Model and Agent


In [ ]:
llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0, max_tokens=2048)

system_prompt = """You are a helpful medical information assistant. You have access to a database of medical documents.

When answering questions:
1. ALWAYS search the document database first using the search_documents tool with a query parameter
2. The search_documents tool takes two parameters: query (required) and top_k (optional, default 3)
3. CITE the sources you find
4. If the documents don't contain relevant information, say so
5. NEVER make up medical information
6. Include relevant information from multiple documents when appropriate
7. For drug interactions or safety concerns, be especially careful and cite sources

Your goal is to provide accurate, well-sourced medical information."""

agent = create_agent(model=llm, tools=[search_documents], system_prompt=system_prompt)


### Running the Agent

#### Demo the agent with example queries

In [ ]:
print("\n" + "=" * 70)
print("Demo")
print("=" * 70)

demo_queries = [
    "Can I take aspirin and ibuprofen together?",
    "What are the alternatives to NSAIDs for pain management?",
    "How should I handle an NSAID overdose?",
]

for i, query in enumerate(demo_queries, 1):
    print(f"\n[Query {i}] {query}")

    try:
        result = agent.invoke({"messages": [HumanMessage(content=query)]})

        answer = result["messages"][-1].content
        print(f"Agent>\n{answer}\n")

    except Exception as e:
        print(f"Error: {e}\n")


#### Interactive RAG System

In [ ]:
def run_medical_rag_agent():
    """Run the agentic RAG system in interactive mode."""

    print("\n" + "=" * 70)
    print("AGENTIC RAG SYSTEM - Medical Information Assistant")
    print("=" * 70)
    print("\nI have medical documents. Ask me anything about medications.")
    print("\nExamples:")
    print("  - 'What are the side effects of ibuprofen?'")
    print("  - 'Can I take aspirin and ibuprofen together?'")
    print("  - 'What are alternatives to NSAIDs for pain management?'")
    print("  - 'How should I handle an NSAID overdose?'")
    print("=" * 70)

    query_count = 0

    while True:
        try:
            user_input = input("\nYour question: ").strip()

            query_count += 1
            print("\nSearching documents and thinking...")

            try:
                result = agent.invoke({"messages": [HumanMessage(content=user_input)]})

                final_answer = result["messages"][-1].content
                print(f"\nAssistant>\n{final_answer}")

            except Exception as agent_error:
                error_msg = f"Error processing query: {str(agent_error)}"
                print(f"\nerror: {error_msg}")
                print("Tip: Try rephrasing your question or use simpler language.")

        except KeyboardInterrupt:
            print("\n\nInterrupted by user.")
            break
        except Exception as e:
            print(f"\nUnexpected error: {str(e)}")
            print("Please try again.")


In [ ]:
import ipywidgets as widgets
from IPython.display import display

text = widgets.Text(placeholder="Ask away!")
send = widgets.Button(description="Send")
out = widgets.Output()
box_layout = widgets.Layout(
    overflow="scroll hidden",
    width="100%",
    flex_flow="row",
    display="flex",
)

print("\n" + "=" * 70)
print("Interactive Mode")
print("=" * 70)


def on_click(b):
    query = text.value.strip()
    if not query:
        return
    text.value = ""  # clear input
    with out:
        try:
            run_medical_rag_agent()
        except Exception as e:
            print("Error:", e)


send.on_click(on_click)
display(
    widgets.VBox(
        [text, send, widgets.Box([widgets.Label("Responses:"), out], layout=box_layout)]
    )
)
